In [1]:
%load_ext autoreload
%autoreload 2

Try classical SVM

In [2]:
#scikit learn and tools
import pandas as pd
import numpy as np;
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from embedding import kernel_matrix

In [3]:
#load dataset with panda
#data are scaled outside the notebook
env = pd.read_csv('env.sel2.scaled.csv')  

In [4]:
#take a look of the dataset
env.describe()

,illuminance,blinds,lamps,co,rh,co2,temp,occupancy
count,1.093400e+04,10934.000000,10934.000000,1.093400e+04,10934.000000,1.093400e+04,1.093400e+04,10934.000000
mean,2.859327e-17,0.803111,0.504677,-1.871559e-16,0.471725,1.455657e-16,-4.159021e-16,-0.315164
std,1.000000e+00,0.371107,0.498983,1.000000e+00,0.076689,1.000000e+00,1.000000e+00,0.949081
min,-9.310658e-01,0.000000,0.000000,-8.039283e-01,0.243300,-1.145063e+00,-1.711094e+00,-1.000000
25%,-8.848906e-01,1.000000,0.000000,-7.808253e-01,0.431300,-7.923978e-01,-8.078199e-01,-1.000000
50%,2.542024e-02,1.000000,1.000000,-2.725584e-01,0.487400,-3.964529e-01,-1.153100e-01,-1.000000
75%,3.882253e-01,1.000000,1.000000,3.743267e-01,0.519800,6.966369e-01,7.428010e-01,1.000000
max,9.814560e+00,1.000000,1.000000,6.542838e+00,0.633300,3.950372e+00,2.594513e+00,1.000000


In [5]:
#DEFINE design matrix
Y = env['occupancy']
#X = env[['illuminance', 'blinds','lamps','co', 'rh', 'co2', 'temp']]
X = env[['illuminance', 'blinds','lamps','rh', 'co2', 'temp']]

X.head()

,illuminance,blinds,lamps,rh,co2,temp
0,-0.90468,1.0,0.0,0.4069,-0.910870,-0.030001
1,-0.90468,1.0,0.0,0.4069,-0.910870,-0.030001
2,-0.90468,1.0,0.0,0.4069,-0.910870,-0.030001
3,-0.90468,1.0,0.0,0.4069,-0.795139,-0.030001
4,-0.90468,1.0,0.0,0.4069,-0.795139,-0.030001


In [6]:
#split design matrix (25% of the design matrix used for test)
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [7]:
#sanity check
print('TRAIN*****')
display(X_train.head())
display(X_train.describe())
print('TEST*****')
display(X_test.head())
display(X_test.describe())

TRAIN*****


,illuminance,blinds,lamps,rh,co2,temp
5491,0.150753,1.00,1.0,0.4941,1.367795,0.742801
1654,1.087450,1.00,1.0,0.3934,-0.304863,0.356400
4448,0.223314,0.14,1.0,0.5033,2.089985,-0.807820
1212,0.467383,0.06,1.0,0.3464,-0.700365,2.293422
5276,0.751030,1.00,1.0,0.4990,-0.400098,-0.677347


,illuminance,blinds,lamps,rh,co2,temp
count,8200.000000,8200.000000,8200.000000,8200.000000,8200.000000,8200.000000
mean,-0.007347,0.800801,0.504083,0.471928,-0.000643,-0.002386
std,0.988695,0.372189,0.499103,0.076101,0.999592,1.001724
min,-0.931066,0.000000,0.000000,0.243300,-1.145063,-1.711094
25%,-0.884891,1.000000,0.000000,0.431300,-0.791494,-0.807820
50%,0.025420,1.000000,1.000000,0.486800,-0.396905,-0.115310
75%,0.371734,1.000000,1.000000,0.519800,0.698455,0.742801
max,9.814560,1.000000,1.000000,0.633300,3.950372,2.594513


TEST*****


,illuminance,blinds,lamps,rh,co2,temp
5934,-0.752961,1.00,0.0,0.4905,-0.680764,-1.535458
7940,-0.931066,1.00,0.0,0.5314,-0.189584,-0.331091
1495,-0.917873,0.06,0.0,0.3831,-0.884892,-0.160473
9165,-0.931066,1.00,0.0,0.6193,0.369940,-0.030001
9060,0.032017,1.00,1.0,0.6138,0.534433,-0.115310


,illuminance,blinds,lamps,rh,co2,temp
count,2734.000000,2734.000000,2734.000000,2734.000000,2734.000000,2734.000000
mean,0.022037,0.810037,0.506459,0.471116,0.001928,0.007156
std,1.033038,0.367823,0.498711,0.078435,1.001403,0.994961
min,-0.931066,0.000000,0.000000,0.243300,-1.145063,-1.711094
25%,-0.878294,1.000000,0.000000,0.426250,-0.797413,-0.807820
50%,0.032017,1.000000,1.000000,0.489300,-0.374120,-0.160473
75%,0.394822,1.000000,1.000000,0.521600,0.669293,0.787965
max,7.169381,1.000000,1.000000,0.633300,3.950372,2.594513


In [8]:
#WARNING: convert data to numpy. Quantum stuff (Qiskit) do not like PANDAS
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()


In [9]:
#define data for training
K = 150
K_test = 50

In [10]:
#try SVM using RBF kernel
svm = SVC(kernel='rbf').fit(X_train_np[:K], y_train_np[:K]);
#result...
predictions = svm.predict(X_test_np[:K_test])
accuracy_score(predictions, y_test[:K_test])

0.82

Try quantum Kernel Method

In [11]:
#Create a cotainer
from embedding import CircuitContainer
from embedding import QEmbedding

c = CircuitContainer(f_embedding= QEmbedding.createQuantumEmbedding, nwire=6, obs=['ZIIIII', 'ZIIZII','ZZZZZZ', 'IZIIII','ZZIIII','ZZIIZI'])

*** Create a Container ***
*** Created quantum template for feature map using 6 qubit ***
     ┌───┐┌───────────┐                         ┌───┐ ░ ┌───┐
q_0: ┤ H ├┤ Rz(phi_0) ├──■──────────────────────┤ X ├─░─┤ H ├
     ├───┤├───────────┤┌─┴─┐                    └─┬─┘ ░ ├───┤
q_1: ┤ H ├┤ Rz(phi_1) ├┤ X ├──■───────────────────┼───░─┤ H ├
     ├───┤├───────────┤└───┘┌─┴─┐                 │   ░ ├───┤
q_2: ┤ H ├┤ Rz(phi_2) ├─────┤ X ├──■──────────────┼───░─┤ H ├
     ├───┤├───────────┤     └───┘┌─┴─┐            │   ░ ├───┤
q_3: ┤ H ├┤ Rz(phi_3) ├──────────┤ X ├──■─────────┼───░─┤ H ├
     ├───┤├───────────┤          └───┘┌─┴─┐       │   ░ ├───┤
q_4: ┤ H ├┤ Rz(phi_4) ├───────────────┤ X ├──■────┼───░─┤ H ├
     ├───┤├───────────┤               └───┘┌─┴─┐  │   ░ ├───┤
q_5: ┤ H ├┤ Rz(phi_5) ├────────────────────┤ X ├──■───░─┤ H ├
     └───┘└───────────┘                    └───┘      ░ └───┘
*** Required observables: ['ZIIIII', 'ZIIZII', 'ZZZZZZ', 'IZIIII', 'ZZIIII', 'ZZIIZI']


In [12]:
#take info about the current container
c.metadata()

*** Quantum template for feature map using 6 qubit ***
     ┌───┐┌───────────┐                         ┌───┐ ░ ┌───┐
q_0: ┤ H ├┤ Rz(phi_0) ├──■──────────────────────┤ X ├─░─┤ H ├
     ├───┤├───────────┤┌─┴─┐                    └─┬─┘ ░ ├───┤
q_1: ┤ H ├┤ Rz(phi_1) ├┤ X ├──■───────────────────┼───░─┤ H ├
     ├───┤├───────────┤└───┘┌─┴─┐                 │   ░ ├───┤
q_2: ┤ H ├┤ Rz(phi_2) ├─────┤ X ├──■──────────────┼───░─┤ H ├
     ├───┤├───────────┤     └───┘┌─┴─┐            │   ░ ├───┤
q_3: ┤ H ├┤ Rz(phi_3) ├──────────┤ X ├──■─────────┼───░─┤ H ├
     ├───┤├───────────┤          └───┘┌─┴─┐       │   ░ ├───┤
q_4: ┤ H ├┤ Rz(phi_4) ├───────────────┤ X ├──■────┼───░─┤ H ├
     ├───┤├───────────┤               └───┘┌─┴─┐  │   ░ ├───┤
q_5: ┤ H ├┤ Rz(phi_5) ├────────────────────┤ X ├──■───░─┤ H ├
     └───┘└───────────┘                    └───┘      ░ └───┘
*** Required observables: ['ZIIIII', 'ZIIZII', 'ZZZZZZ', 'IZIIII', 'ZZIIII', 'ZZIIZI']


In [16]:
#IF NEEDED, REBUILD THE CONTAINER
c.build(f_embedding= QEmbedding.createQuantumEmbedding, nwire= 6, obs=['ZIIIII', 'ZIIZII','ZZZZZZ', 'IZIIII','ZZIIII','ZZIIZI'])

#c.build(f_embedding= QEmbedding.createQuantumEmbedding, nwire= 6, obs=['ZIIIII'])

*** Created quantum template for feature map using 6 qubit ***
     ┌───┐┌───────────┐                         ┌───┐ ░ ┌───┐
q_0: ┤ H ├┤ Rz(phi_0) ├──■──────────────────────┤ X ├─░─┤ H ├
     ├───┤├───────────┤┌─┴─┐                    └─┬─┘ ░ ├───┤
q_1: ┤ H ├┤ Rz(phi_1) ├┤ X ├──■───────────────────┼───░─┤ H ├
     ├───┤├───────────┤└───┘┌─┴─┐                 │   ░ ├───┤
q_2: ┤ H ├┤ Rz(phi_2) ├─────┤ X ├──■──────────────┼───░─┤ H ├
     ├───┤├───────────┤     └───┘┌─┴─┐            │   ░ ├───┤
q_3: ┤ H ├┤ Rz(phi_3) ├──────────┤ X ├──■─────────┼───░─┤ H ├
     ├───┤├───────────┤          └───┘┌─┴─┐       │   ░ ├───┤
q_4: ┤ H ├┤ Rz(phi_4) ├───────────────┤ X ├──■────┼───░─┤ H ├
     ├───┤├───────────┤               └───┘┌─┴─┐  │   ░ ├───┤
q_5: ┤ H ├┤ Rz(phi_5) ├────────────────────┤ X ├──■───░─┤ H ├
     └───┘└───────────┘                    └───┘      ░ └───┘
*** Required observables: ['ZIIIII']


In [17]:
#try SVM using linear kernel (dot product)
svm_quantum = SVC(kernel=kernel_matrix).fit(X_train_np[:K], y_train_np[:K]);

In [18]:
q_predictions = svm_quantum.predict(X_test_np[:K_test])
accuracy_score(q_predictions, y_test_np[:K_test])

0.64